In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
openai_api_key="your OPENAI API key"

# Format Dataset

In [ ]:
import pandas as pd
responses_path="/content/drive/MyDrive/Arunadiss/main/Evaluation/Data/Results/qualitative_data_questions/concat_dfs/model_responses.csv"

ds_responses=pd.read_csv(responses_path)


In [ ]:
import itertools
import pandas as pd

# Define a list of all the model columns
model_columns = [
    'gpt-4 Answer', 'gpt-3.5-turbo Answer', 'RAG gpt-4 Answer', 'RAG gpt-3.5-turbo Answer',
    'Vanilla Mistral Answer', 'Fine Tuned Mistral Answer', 'Vanilla Biomistral DARE Answer',
    'Fine Tuned Biomistral DARE Answer'
]

matchups = []

# Iterate through each row (each question)
for idx, row in ds_responses.iterrows():
    question = row['Qualitative benchmark question']
    markscheme = row['Markscheme']

    # Create all possible pairings for the models
    for model1, model2 in itertools.combinations(model_columns, 2):
        matchups.append({
            'Question': question,
            'Markscheme': markscheme,
            'Model 1': model1,
            'Model 1 Response': row[model1],
            'Model 2': model2,
            'Model 2 Response': row[model2],
            'Win': 0,  # Initialize to 0; 1 if Model 1 wins
            'Tie': 0,  # Initialize to 0; 1 if it's a tie
            'Loss': 0,  # Initialize to 0; 1 if Model 1 loses
            'LLM Evaluator Reasoning': ''  # Empty string to be filled with evaluator's reasoning
        })

# Convert the list of matchups into a DataFrame
matchups_df = pd.DataFrame(matchups)


In [ ]:
matchups_df

,Question,Markscheme,Model 1,Model 1 Response,Model 2,Model 2 Response,Win,Tie,Loss,LLM Evaluator Reasoning
0,What are the characteristics of stem cells tha...,- (stem cells) have/retain the capacity to div...,gpt-4 Answer,Stem cells possess several characteristics tha...,gpt-3.5-turbo Answer,Stem cells have the following characteristics ...,0,0,0,
1,What are the characteristics of stem cells tha...,- (stem cells) have/retain the capacity to div...,gpt-4 Answer,Stem cells possess several characteristics tha...,RAG gpt-4 Answer,Stem cells possess several key characteristics...,0,0,0,
2,What are the characteristics of stem cells tha...,- (stem cells) have/retain the capacity to div...,gpt-4 Answer,Stem cells possess several characteristics tha...,RAG gpt-3.5-turbo Answer,Stem cells possess several key characteristics...,0,0,0,
3,What are the characteristics of stem cells tha...,- (stem cells) have/retain the capacity to div...,gpt-4 Answer,Stem cells possess several characteristics tha...,Vanilla Mistral Answer,Stem cells have the ability to differentiate i...,0,0,0,
4,What are the characteristics of stem cells tha...,- (stem cells) have/retain the capacity to div...,gpt-4 Answer,Stem cells possess several characteristics tha...,Fine Tuned Mistral Answer,Stem cells have several unique properties that...,0,0,0,
...,...,...,...,...,...,...,...,...,...,...
975,Can you explain the process of spermatogenesis...,- production of sperm/spermatozoa in the teste...,Vanilla Mistral Answer,Spermatogenesis is the process by which male g...,Vanilla Biomistral DARE Answer,Spermatogenesis is the process by which haploi...,0,0,0,
976,Can you explain the process of spermatogenesis...,- production of sperm/spermatozoa in the teste...,Vanilla Mistral Answer,Spermatogenesis is the process by which male g...,Fine Tuned Biomistral DARE Answer,"Certainly! In humans, spermatogenesis is the p...",0,0,0,
977,Can you explain the process of spermatogenesis...,- production of sperm/spermatozoa in the teste...,Fine Tuned Mistral Answer,Sure! Spermatogenesis is the process through w...,Vanilla Biomistral DARE Answer,Spermatogenesis is the process by which haploi...,0,0,0,
978,Can you explain the process of spermatogenesis...,- production of sperm/spermatozoa in the teste...,Fine Tuned Mistral Answer,Sure! Spermatogenesis is the process through w...,Fine Tuned Biomistral DARE Answer,"Certainly! In humans, spermatogenesis is the p...",0,0,0,


As we can see the number of games in total equate to 980.

In [ ]:
match_ups_path="/content/drive/MyDrive/Arunadiss/main/Evaluation/Data/Results/Chatbot_Arena_Data/match_ups.csv"

matchups_df.to_csv(match_ups_path, index=False)

# ChatbotArena

## Use GPT 4 as evaluator

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 5.2 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI

client = OpenAI(api_key=openai_api_key)


In [ ]:
from openai import OpenAI

client = OpenAI(api_key=openai_api_key)

def answer(evaluator_answer,model_):
    # Define the new task for the assistant

    prompt = (
    f"Evaluator's Assessment: {evaluator_answer}\n\n"
    "Your task is to classify the outcome of this assessment focusing specifically on Teacher 1's performance compared to Teacher 2. Use the following classes:\n"
    "- Class '2': Teacher 1 was better than Teacher 2 (indicates a superior performance by Teacher 1).\n"
    "- Class '1': There was a tie between Teacher 1 and Teacher 2 (indicates equivalent performance).\n"
    "- Class '0': Teacher 1 was worse than Teacher 2 (indicates a superior performance by Teacher 2, thus worse performance by Teacher 1).\n"
    "Based on the evaluator’s assessment provided above, select JUST the appropriate class number as your answer."
)
    system_message = (
    "You are an assistant tasked with classifying an evaluator’s assessment regarding a comparison between two teachers. "
    "Focus on Teacher 1's performance in relation to Teacher 2. You are not to make any independent evaluations but should classify based on the provided assessment. "
    "Determine and classify whether Teacher 1 was better than, tied with, or worse than Teacher 2 based on the assessment. "
    "Use the class numbers to indicate your classification: 2 for better, 1 for tied, 0 for worse."
)

    # Use OpenAI API to generate the response
    response = client.chat.completions.create(
        model=model_,
        messages=[
            {
                "role": "system",
                "content": system_message
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature=0.3,
        max_tokens=20,
        top_p=1
    )

    complete_response = response.choices[0].message.content.strip()
    # Return the generated response
    return complete_response.strip()


In [ ]:
import pandas as pd
llama_3_8b_df=pd.read_csv("/content/drive/MyDrive/Arunadiss/main/Evaluation/Data/Results/Chatbot_Arena_Data/llama 3 8B/llama_3_8b_match_up_results.csv")

In [ ]:
import pandas as pd
match_ups_path="/content/drive/MyDrive/Arunadiss/main/Evaluation/Data/Results/Chatbot_Arena_Data/match_ups.csv"

matchups_df=pd.read_csv(match_ups_path)



In [ ]:

matchups_df

,Question,Markscheme,Model 1,Model 1 Response,Model 2,Model 2 Response,Win,Tie,Loss,LLM Evaluator Reasoning
0,What are the characteristics of stem cells tha...,- (stem cells) have/retain the capacity to div...,gpt-4 Answer,Stem cells possess several characteristics tha...,gpt-3.5-turbo Answer,Stem cells have the following characteristics ...,0,0,0,NaN
1,What are the characteristics of stem cells tha...,- (stem cells) have/retain the capacity to div...,gpt-4 Answer,Stem cells possess several characteristics tha...,RAG gpt-4 Answer,Stem cells possess several key characteristics...,0,0,0,NaN
2,What are the characteristics of stem cells tha...,- (stem cells) have/retain the capacity to div...,gpt-4 Answer,Stem cells possess several characteristics tha...,RAG gpt-3.5-turbo Answer,Stem cells possess several key characteristics...,0,0,0,NaN
3,What are the characteristics of stem cells tha...,- (stem cells) have/retain the capacity to div...,gpt-4 Answer,Stem cells possess several characteristics tha...,Vanilla Mistral Answer,Stem cells have the ability to differentiate i...,0,0,0,NaN
4,What are the characteristics of stem cells tha...,- (stem cells) have/retain the capacity to div...,gpt-4 Answer,Stem cells possess several characteristics tha...,Fine Tuned Mistral Answer,Stem cells have several unique properties that...,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...
975,Can you explain the process of spermatogenesis...,- production of sperm/spermatozoa in the teste...,Vanilla Mistral Answer,Spermatogenesis is the process by which male g...,Vanilla Biomistral DARE Answer,Spermatogenesis is the process by which haploi...,0,0,0,NaN
976,Can you explain the process of spermatogenesis...,- production of sperm/spermatozoa in the teste...,Vanilla Mistral Answer,Spermatogenesis is the process by which male g...,Fine Tuned Biomistral DARE Answer,"Certainly! In humans, spermatogenesis is the p...",0,0,0,NaN
977,Can you explain the process of spermatogenesis...,- production of sperm/spermatozoa in the teste...,Fine Tuned Mistral Answer,Sure! Spermatogenesis is the process through w...,Vanilla Biomistral DARE Answer,Spermatogenesis is the process by which haploi...,0,0,0,NaN
978,Can you explain the process of spermatogenesis...,- production of sperm/spermatozoa in the teste...,Fine Tuned Mistral Answer,Sure! Spermatogenesis is the process through w...,Fine Tuned Biomistral DARE Answer,"Certainly! In humans, spermatogenesis is the p...",0,0,0,NaN


In [ ]:
example_q_1=matchups_df['Question'][1]
example_q_1_ms=matchups_df['Markscheme'][1]
example_r_1=matchups_df['Model 1 Response'][1]
example_r_2=matchups_df['Model 2 Response'][1]


In [ ]:
def generate_prompt(question, markscheme, response1, response2):
    prompt = f"""
You are an AI tasked with evaluating two teacher responses to a student's question.
Alongside a markscheme, which lists key points that should be included in an ideal response,
you should also assess how well each teacher embodies the principles of the INSPIRE framework.
The INSPIRE framework values teachers being:
- Intelligent: Responses should adhere closely to the markscheme, demonstrating knowledge and accuracy.
- Nurturant: Teachers should show understanding and support for learners' difficulties.
- Socratic: Teachers should ask questions that stimulate critical thinking rather than just providing answers.
- Progressive: Responses should build on what the learner knows, gradually introducing more complex ideas.
- Indirect: Teachers should guide learners to find answers themselves, promoting independent problem-solving.
- Reflective: Teachers should encourage learners to think about their own learning processes and understand their mistakes.
- Encouraging: Teachers should motivate learners, highlighting their progress and achievements.

Please determine whether Teacher 1's response or Teacher 2's response better demonstrates these teaching characteristics, and how well each aligns with the markscheme.

Question: {question}

Markscheme:
{markscheme}

Response from Teacher 1:
{response1}

Response from Teacher 2:
{response2}

Based on the question, the markscheme, and the INSPIRE teaching principles, decide whether Teacher 1's response is better, equal, or worse than Teacher 2's response.
Think step by step, and at the end make a final decision amongst these three options.
    """
    return prompt

prompt_example=generate_prompt(example_q_1,example_q_1_ms,example_r_1,example_r_2)
print(prompt_example)



You are an AI tasked with evaluating two teacher responses to a student's question.
Alongside a markscheme, which lists key points that should be included in an ideal response,
you should also assess how well each teacher embodies the principles of the INSPIRE framework.
The INSPIRE framework values teachers being:
- Intelligent: Responses should adhere closely to the markscheme, demonstrating knowledge and accuracy.
- Nurturant: Teachers should show understanding and support for learners' difficulties.
- Socratic: Teachers should ask questions that stimulate critical thinking rather than just providing answers.
- Progressive: Responses should build on what the learner knows, gradually introducing more complex ideas.
- Indirect: Teachers should guide learners to find answers themselves, promoting independent problem-solving.
- Reflective: Teachers should encourage learners to think about their own learning processes and understand their mistakes.
- Encouraging: Teachers should motivat

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=openai_api_key)


def llm_evaluator(question, markscheme, response1, response2):
    system_message=f"""
You are an AI tasked with evaluating two teacher responses to a student's question.
Alongside a markscheme, which lists key points that should be included in an ideal response,
you should also assess how well each teacher embodies the principles of the INSPIRE framework.
The INSPIRE framework values teachers being:
- Intelligent: Responses should adhere closely to the markscheme, demonstrating knowledge and accuracy.
- Nurturant: Teachers should show understanding and support for learners' difficulties.
- Socratic: Teachers should ask questions that stimulate critical thinking rather than just providing answers.
- Progressive: Responses should build on what the learner knows, gradually introducing more complex ideas.
- Indirect: Teachers should guide learners to find answers themselves, promoting independent problem-solving.
- Reflective: Teachers should encourage learners to think about their own learning processes and understand their mistakes.
- Encouraging: Teachers should motivate learners, highlighting their progress and achievements.

Please determine whether Teacher 1's response or Teacher 2's response better demonstrates these teaching characteristics, and how well each aligns with the markscheme.
"""
    prompt=generate_prompt(question, markscheme, response1, response2)

    # Use OpenAI API to generate the response
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {
                "role": "system",
                "content": system_message
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature=0.7,
        max_tokens=600,
        top_p=1
    )

    complete_response = response.choices[0].message.content.strip()
    # Return the generated response
    return complete_response

def generate_prompt(question, markscheme, response1, response2):
    prompt = f"""

Question: {question}

Markscheme:
{markscheme}

Response from Teacher 1:
{response1}

Response from Teacher 2:
{response2}

Based on the question, the markscheme, and the INSPIRE teaching principles, decide whether Teacher 1's response is better, equal, or worse than Teacher 2's response.
Think step by step, and at the end make a clear final decision amongst these three options.
    """
    return prompt

In [ ]:
example=llm_evaluator(example_q_1,example_q_1_ms,example_r_1,example_r_2)
print(example)

Both teachers provide comprehensive and accurate responses that align well with the markscheme. They highlight the key characteristics of stem cells - their ability to divide, differentiate, and potentially treat diseases.

Teacher 1's response adheres closely to the markscheme, demonstrating intelligence. The teacher uses a nurturing approach, explaining complex concepts in a clear and understandable manner. They are progressive, building on simpler ideas to introduce more complex applications of stem cells, like disease modeling and personalized medicine. However, the response lacks in Socratic, indirect, and reflective aspects as the teacher did not ask probing questions or encourage the student to think about the information independently.

Teacher 2's response also aligns with the markscheme and exhibits intelligence. The teacher demonstrates a nurturing approach by clearly explaining the different types of potency. The response is progressive, explaining how stem cells can be use

In [ ]:
from tqdm import tqdm
import time
delay_duration=10
gpt_4_eval_df=matchups_df
# Iterate over the DataFrame and evaluate each head-to-head matchup
for index, row in tqdm(gpt_4_eval_df.iterrows(), total=gpt_4_eval_df.shape[0], position=0, leave=True):
    evaluation_result = llm_evaluator(
        row['Question'],
        row['Markscheme'],
        row['Model 1 Response'],
        row['Model 2 Response']
    )
    # Update the DataFrame directly with the LLM evaluator reasoning
    gpt_4_eval_df.at[index, 'LLM Evaluator Reasoning'] = evaluation_result

    # Save the DataFrame to CSV every 10 iterations
    if index % 10 == 0:
        gpt_4_eval_df.to_csv("/content/drive/MyDrive/Arunadiss/main/Evaluation/Data/Results/Chatbot_Arena_Data/gpt 4/gpt_4_match_up_results.csv",index=False)
    if index % 5 == 0:
        # Wait for a specified number of seconds before making the next request
        time.sleep(delay_duration)
gpt_4_eval_df.to_csv("/content/drive/MyDrive/Arunadiss/main/Evaluation/Data/Results/Chatbot_Arena_Data/gpt 4/gpt_4_match_up_results.csv",index=False)


100%|██████████| 980/980 [4:55:17<00:00, 18.08s/it]


In [ ]:
gpt_4_eval_df


,Question,Markscheme,Model 1,Model 1 Response,Model 2,Model 2 Response,Win,Tie,Loss,LLM Evaluator Reasoning
0,What are the characteristics of stem cells tha...,- (stem cells) have/retain the capacity to div...,gpt-4 Answer,Stem cells possess several characteristics tha...,gpt-3.5-turbo Answer,Stem cells have the following characteristics ...,0,0,0,Both teachers provided comprehensive responses...
1,What are the characteristics of stem cells tha...,- (stem cells) have/retain the capacity to div...,gpt-4 Answer,Stem cells possess several characteristics tha...,RAG gpt-4 Answer,Stem cells possess several key characteristics...,0,0,0,Teacher 1's response:\n\nIntelligent: The resp...
2,What are the characteristics of stem cells tha...,- (stem cells) have/retain the capacity to div...,gpt-4 Answer,Stem cells possess several characteristics tha...,RAG gpt-3.5-turbo Answer,Stem cells possess several key characteristics...,0,0,0,Looking at each teacher's response in relation...
3,What are the characteristics of stem cells tha...,- (stem cells) have/retain the capacity to div...,gpt-4 Answer,Stem cells possess several characteristics tha...,Vanilla Mistral Answer,Stem cells have the ability to differentiate i...,0,0,0,Considering the INSPIRE framework and the mark...
4,What are the characteristics of stem cells tha...,- (stem cells) have/retain the capacity to div...,gpt-4 Answer,Stem cells possess several characteristics tha...,Fine Tuned Mistral Answer,Stem cells have several unique properties that...,0,0,0,Both Teacher 1 and Teacher 2 provide detailed ...
...,...,...,...,...,...,...,...,...,...,...
975,Can you explain the process of spermatogenesis...,- production of sperm/spermatozoa in the teste...,Vanilla Mistral Answer,Spermatogenesis is the process by which male g...,Vanilla Biomistral DARE Answer,Spermatogenesis is the process by which haploi...,0,0,0,"Looking at the markscheme, both Teacher 1 and ..."
976,Can you explain the process of spermatogenesis...,- production of sperm/spermatozoa in the teste...,Vanilla Mistral Answer,Spermatogenesis is the process by which male g...,Fine Tuned Biomistral DARE Answer,"Certainly! In humans, spermatogenesis is the p...",0,0,0,Let's start by comparing each response to the ...
977,Can you explain the process of spermatogenesis...,- production of sperm/spermatozoa in the teste...,Fine Tuned Mistral Answer,Sure! Spermatogenesis is the process through w...,Vanilla Biomistral DARE Answer,Spermatogenesis is the process by which haploi...,0,0,0,Looking at the markscheme first:\n\nBoth Teach...
978,Can you explain the process of spermatogenesis...,- production of sperm/spermatozoa in the teste...,Fine Tuned Mistral Answer,Sure! Spermatogenesis is the process through w...,Fine Tuned Biomistral DARE Answer,"Certainly! In humans, spermatogenesis is the p...",0,0,0,Both responses adhere closely to the markschem...


## Process responses

### Gpt 4 processing

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 5.2 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI

client = OpenAI(api_key=openai_api_key)


In [ ]:

def answer(evaluator_answer,model_):
    # Define the new task for the assistant

    prompt = (

    "Your task is to classify the outcome of this assessment focusing specifically on Teacher 1's performance compared to Teacher 2. Use the following classes:\n"
    "- Class '2': Teacher 1 was better than Teacher 2 (indicates a superior performance by Teacher 1).\n"
    "- Class '1': There was a tie between Teacher 1 and Teacher 2 (indicates equivalent performance).\n"
    "- Class '0': Teacher 1 was worse than Teacher 2 (indicates a superior performance by Teacher 2, thus worse performance by Teacher 1).\n"
    "Based on the evaluator’s assessment provided below, select JUST the appropriate class number as your answer."
    f"Evaluator's Assessment: {evaluator_answer}\n\n"
    "Class:"
)
    system_message = (
    "You are an assistant tasked with classifying an evaluator’s assessment regarding a comparison between two teachers. "
    "Focus on Teacher 1's performance in relation to Teacher 2. You are not to make any independent evaluations but should classify based on the provided assessment. "
    "Determine and classify whether Teacher 1 was better than, tied with, or worse than Teacher 2 based on the assessment. "
    "Use the class numbers to indicate your classification: 2 for better, 1 for tied, 0 for worse."

)

    # Use OpenAI API to generate the response
    response = client.chat.completions.create(
        model=model_,
        messages=[
            {
                "role": "system",
                "content": system_message
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature=0.3,
        max_tokens=20,
        top_p=1
    )

    complete_response = response.choices[0].message.content.strip()
    # Return the generated response
    return complete_response.strip()


In [ ]:
import pandas as pd
gpt_4_eval_df=pd.read_csv("/content/drive/MyDrive/Arunadiss/main/Evaluation/Data/Results/Chatbot_Arena_Data/gpt 4/gpt_4_match_up_results.csv")

In [ ]:
gpt_4_eval_df['LLM Evaluator Reasoning'][80]

"Let's evaluate the responses of both teachers according to the markscheme and INSPIRE principles:\n\nMarkscheme:\n- Both teachers mention the phospholipid bilayer and the orientation of the hydrophilic heads and hydrophobic tails.\n- Teacher 2 specifically mentions integral and peripheral proteins, which aligns with the markscheme. Teacher 1 does not.\n- Neither teacher mentions protein channels, glycoproteins with carbohydrates attached, cholesterol, or the thickness of the membrane. \n\nINSPIRE Principles:\n- Intelligent: Both teachers demonstrate knowledge and accuracy in their responses, although Teacher 2's response is more aligned with the markscheme.\n- Nurturant: Neither teacher explicitly shows understanding or support for potential learner difficulties.\n- Socratic: Neither teacher asks questions to stimulate critical thinking.\n- Progressive: Both teachers build on basic knowledge of the plasma membrane's structure. \n- Indirect: Neither teacher guides the learner to find t

In [ ]:
gpt_4_eval_df['Model 2'][80]

'Fine Tuned Biomistral DARE Answer'

In [ ]:
answer(gpt_4_eval_df['LLM Evaluator Reasoning'][80],'gpt-3.5-turbo')


'2'

In [ ]:
print(gpt_4_eval_df['LLM Evaluator Reasoning'][80])

Let's evaluate the responses of both teachers according to the markscheme and INSPIRE principles:

Markscheme:
- Both teachers mention the phospholipid bilayer and the orientation of the hydrophilic heads and hydrophobic tails.
- Teacher 2 specifically mentions integral and peripheral proteins, which aligns with the markscheme. Teacher 1 does not.
- Neither teacher mentions protein channels, glycoproteins with carbohydrates attached, cholesterol, or the thickness of the membrane. 

INSPIRE Principles:
- Intelligent: Both teachers demonstrate knowledge and accuracy in their responses, although Teacher 2's response is more aligned with the markscheme.
- Nurturant: Neither teacher explicitly shows understanding or support for potential learner difficulties.
- Socratic: Neither teacher asks questions to stimulate critical thinking.
- Progressive: Both teachers build on basic knowledge of the plasma membrane's structure. 
- Indirect: Neither teacher guides the learner to find the answer the

In [ ]:
from tqdm import tqdm

class_evaluation=[answer(question,'gpt-3.5-turbo') for question in tqdm(gpt_4_eval_df['LLM Evaluator Reasoning'], position=0, leave=True)]
gpt_4_eval_df['Classify evaluator response']=class_evaluation


100%|██████████| 980/980 [07:23<00:00,  2.21it/s]


In [ ]:
import pandas as pd
import re
import numpy as np

# Define a function to search for the first valid digit
def find_first_valid_classification(text):
    # Using regular expression to find all digits in the string
    matches = re.findall(r'\b[0-2]\b', text)  # \b for word boundary to ensure single characters
    if matches:
        return int(matches[0])  # Convert the first match to integer and return
    return None  # Return None if no match found

# Apply the function to each element in the column and create a new column for the results
gpt_4_eval_df['Classify evaluator response'] = gpt_4_eval_df['Classify evaluator response'].apply(find_first_valid_classification)

# Check for NaN values in the newly created column
if gpt_4_eval_df['Classify evaluator response'].isna().any():
    print("There are NaN values in the 'Classify evaluator response' column.")
else:
    print("No NaN values found in the 'Classify evaluator response' column.")


No NaN values found in the 'Classify evaluator response' column.


In [ ]:
gpt_4_eval_df

,Question,Markscheme,Model 1,Model 1 Response,Model 2,Model 2 Response,Win,Tie,Loss,LLM Evaluator Reasoning,Classify evaluator response
0,What are the characteristics of stem cells tha...,- (stem cells) have/retain the capacity to div...,gpt-4 Answer,Stem cells possess several characteristics tha...,gpt-3.5-turbo Answer,Stem cells have the following characteristics ...,0,0,0,Both teachers provided comprehensive responses...,0
1,What are the characteristics of stem cells tha...,- (stem cells) have/retain the capacity to div...,gpt-4 Answer,Stem cells possess several characteristics tha...,RAG gpt-4 Answer,Stem cells possess several key characteristics...,0,0,0,Teacher 1's response:\n\nIntelligent: The resp...,0
2,What are the characteristics of stem cells tha...,- (stem cells) have/retain the capacity to div...,gpt-4 Answer,Stem cells possess several characteristics tha...,RAG gpt-3.5-turbo Answer,Stem cells possess several key characteristics...,0,0,0,Looking at each teacher's response in relation...,0
3,What are the characteristics of stem cells tha...,- (stem cells) have/retain the capacity to div...,gpt-4 Answer,Stem cells possess several characteristics tha...,Vanilla Mistral Answer,Stem cells have the ability to differentiate i...,0,0,0,Considering the INSPIRE framework and the mark...,2
4,What are the characteristics of stem cells tha...,- (stem cells) have/retain the capacity to div...,gpt-4 Answer,Stem cells possess several characteristics tha...,Fine Tuned Mistral Answer,Stem cells have several unique properties that...,0,0,0,Both Teacher 1 and Teacher 2 provide detailed ...,2
...,...,...,...,...,...,...,...,...,...,...,...
975,Can you explain the process of spermatogenesis...,- production of sperm/spermatozoa in the teste...,Vanilla Mistral Answer,Spermatogenesis is the process by which male g...,Vanilla Biomistral DARE Answer,Spermatogenesis is the process by which haploi...,0,0,0,"Looking at the markscheme, both Teacher 1 and ...",1
976,Can you explain the process of spermatogenesis...,- production of sperm/spermatozoa in the teste...,Vanilla Mistral Answer,Spermatogenesis is the process by which male g...,Fine Tuned Biomistral DARE Answer,"Certainly! In humans, spermatogenesis is the p...",0,0,0,Let's start by comparing each response to the ...,1
977,Can you explain the process of spermatogenesis...,- production of sperm/spermatozoa in the teste...,Fine Tuned Mistral Answer,Sure! Spermatogenesis is the process through w...,Vanilla Biomistral DARE Answer,Spermatogenesis is the process by which haploi...,0,0,0,Looking at the markscheme first:\n\nBoth Teach...,1
978,Can you explain the process of spermatogenesis...,- production of sperm/spermatozoa in the teste...,Fine Tuned Mistral Answer,Sure! Spermatogenesis is the process through w...,Fine Tuned Biomistral DARE Answer,"Certainly! In humans, spermatogenesis is the p...",0,0,0,Both responses adhere closely to the markschem...,2


In [ ]:

# Now, let's fill in the Win, Tie, Loss columns based on 'Classify evaluator response'
for index, row in gpt_4_eval_df.iterrows():
    # Get the classification from 'Classify evaluator response' column
    classification = row['Classify evaluator response']

    # Based on the classification, update the Win, Tie, Loss columns
    if classification == 2:
        gpt_4_eval_df.at[index, 'Win'] = 1
        gpt_4_eval_df.at[index, 'Tie'] = 0
        gpt_4_eval_df.at[index, 'Loss'] = 0
    elif classification == 1:
        gpt_4_eval_df.at[index, 'Win'] = 0
        gpt_4_eval_df.at[index, 'Tie'] = 1
        gpt_4_eval_df.at[index, 'Loss'] = 0
    elif classification == 0:
        gpt_4_eval_df.at[index, 'Win'] = 0
        gpt_4_eval_df.at[index, 'Tie'] = 0
        gpt_4_eval_df.at[index, 'Loss'] = 1
    else:
        print(f"{index} has an issue")
        continue  # For now, we'll skip updating the row



In [ ]:
gpt_4_eval_df

,Question,Markscheme,Model 1,Model 1 Response,Model 2,Model 2 Response,Win,Tie,Loss,LLM Evaluator Reasoning,Classify evaluator response
0,What are the characteristics of stem cells tha...,- (stem cells) have/retain the capacity to div...,gpt-4 Answer,Stem cells possess several characteristics tha...,gpt-3.5-turbo Answer,Stem cells have the following characteristics ...,0,0,1,Both teachers provided comprehensive responses...,0
1,What are the characteristics of stem cells tha...,- (stem cells) have/retain the capacity to div...,gpt-4 Answer,Stem cells possess several characteristics tha...,RAG gpt-4 Answer,Stem cells possess several key characteristics...,0,0,1,Teacher 1's response:\n\nIntelligent: The resp...,0
2,What are the characteristics of stem cells tha...,- (stem cells) have/retain the capacity to div...,gpt-4 Answer,Stem cells possess several characteristics tha...,RAG gpt-3.5-turbo Answer,Stem cells possess several key characteristics...,0,0,1,Looking at each teacher's response in relation...,0
3,What are the characteristics of stem cells tha...,- (stem cells) have/retain the capacity to div...,gpt-4 Answer,Stem cells possess several characteristics tha...,Vanilla Mistral Answer,Stem cells have the ability to differentiate i...,1,0,0,Considering the INSPIRE framework and the mark...,2
4,What are the characteristics of stem cells tha...,- (stem cells) have/retain the capacity to div...,gpt-4 Answer,Stem cells possess several characteristics tha...,Fine Tuned Mistral Answer,Stem cells have several unique properties that...,1,0,0,Both Teacher 1 and Teacher 2 provide detailed ...,2
...,...,...,...,...,...,...,...,...,...,...,...
975,Can you explain the process of spermatogenesis...,- production of sperm/spermatozoa in the teste...,Vanilla Mistral Answer,Spermatogenesis is the process by which male g...,Vanilla Biomistral DARE Answer,Spermatogenesis is the process by which haploi...,0,1,0,"Looking at the markscheme, both Teacher 1 and ...",1
976,Can you explain the process of spermatogenesis...,- production of sperm/spermatozoa in the teste...,Vanilla Mistral Answer,Spermatogenesis is the process by which male g...,Fine Tuned Biomistral DARE Answer,"Certainly! In humans, spermatogenesis is the p...",0,1,0,Let's start by comparing each response to the ...,1
977,Can you explain the process of spermatogenesis...,- production of sperm/spermatozoa in the teste...,Fine Tuned Mistral Answer,Sure! Spermatogenesis is the process through w...,Vanilla Biomistral DARE Answer,Spermatogenesis is the process by which haploi...,0,1,0,Looking at the markscheme first:\n\nBoth Teach...,1
978,Can you explain the process of spermatogenesis...,- production of sperm/spermatozoa in the teste...,Fine Tuned Mistral Answer,Sure! Spermatogenesis is the process through w...,Fine Tuned Biomistral DARE Answer,"Certainly! In humans, spermatogenesis is the p...",1,0,0,Both responses adhere closely to the markschem...,2


In [ ]:
gpt_4_eval_df.to_csv("/content/drive/MyDrive/Arunadiss/main/Evaluation/Data/Results/Chatbot_Arena_Data/gpt 4/gpt_4_match_up_results.csv",index=False)


# ELO calculation

## Gpt 4

In [ ]:
import pandas as pd
path="/content/drive/MyDrive/Arunadiss/main/Evaluation/Data/Results/Chatbot_Arena_Data/gpt 4/gpt_4_match_up_results.csv"

gpt_4_eval_df=pd.read_csv(path)

In [ ]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm

initial_elo = 1400
K = 16
n_perm = 1000  # Number of permutations/ bootstrap samples


def update_elo(rating_a, rating_b, score_a, score_b, K):
    expected_a = 1 / (1 + 10 ** ((rating_b - rating_a) / 400))
    expected_b = 1 - expected_a
    new_rating_a = rating_a + K * (score_a - expected_a)
    new_rating_b = rating_b + K * (score_b - expected_b)
    return new_rating_a, new_rating_b


def calculate_elo_ratings(dataframe, K, initial_elo):
    models = set(dataframe['Model 1']) | set(dataframe['Model 2'])
    elo_ratings = {model: initial_elo for model in models}
    for index, row in dataframe.iterrows():
        model_1, model_2 = row['Model 1'], row['Model 2']
        score_a = row['Win']
        if score_a==1:
            score_b=0
        else:
            if row['Tie']==1:
                score_a=0.5
                score_b=0.5
            else:
                score_b=1
        new_rating_1, new_rating_2 = update_elo(elo_ratings[model_1], elo_ratings[model_2], score_a, score_b, K)
        elo_ratings[model_1], elo_ratings[model_2] = new_rating_1, new_rating_2
    return elo_ratings

# Bootstrapping
elo_samples = {model: [] for model in models}
for index,_ in tqdm(enumerate(range(n_perm)),total=n_perm,position=0, leave=True):
    sample_df = gpt_4_eval_df.sample(frac=1, replace=True, random_state=index)
    sample_elo_ratings = calculate_elo_ratings(sample_df, K, initial_elo)
    for model, rating in sample_elo_ratings.items():
        elo_samples[model].append(rating)

# Calculating the 95% confidence interval for each model
confidence_intervals = {model: np.percentile(elo_samples[model], [2.5, 97.5]) for model in models}

# Print the results
# Calculate the average Elo rating for each model
average_elos = {model: np.mean(elo_samples[model]) for model in models}

# Sort models by their average Elo rating in descending order
sorted_models = sorted(average_elos.items(), key=lambda item: item[1], reverse=True)

print("\n\n")
# Display the sorted and ranked models with their confidence intervals
print("Sorted and ranked models with Elo ratings and confidence intervals:")
for rank, (model, elo) in enumerate(sorted_models, start=1):
    interval = confidence_intervals[model]
    print(f"{rank}. {model}: Elo = {elo:.0f}, CI = [{interval[0]:.0f}, {interval[1]:.0f}]")



100%|██████████| 1000/1000 [01:05<00:00, 15.28it/s]




Sorted and ranked models with Elo ratings and confidence intervals:
1. RAG gpt-4 Answer: Elo = 1765, CI = [1704, 1824]
2. RAG gpt-3.5-turbo Answer: Elo = 1648, CI = [1593, 1703]
3. gpt-4 Answer: Elo = 1580, CI = [1526, 1633]
4. gpt-3.5-turbo Answer: Elo = 1380, CI = [1322, 1441]
5. Fine Tuned Mistral Answer: Elo = 1288, CI = [1226, 1347]
6. Fine Tuned Biomistral DARE Answer: Elo = 1255, CI = [1200, 1312]
7. Vanilla Biomistral DARE Answer: Elo = 1184, CI = [1130, 1243]
8. Vanilla Mistral Answer: Elo = 1100, CI = [1042, 1164]


In [ ]:
# Calculate the total matches, wins, ties, and losses for each model
model_stats = {model: {'wins': 0, 'ties': 0, 'losses': 0, 'total': 0} for model in models}

for index, row in gpt_4_eval_df.iterrows():
    model_1 = row['Model 1']
    model_2 = row['Model 2']

    model_stats[model_1]['total'] += 1
    model_stats[model_2]['total'] += 1

    if row['Win'] == 1:
        model_stats[model_1]['wins'] += 1
        model_stats[model_2]['losses'] += 1
    elif row['Tie'] == 1:
        model_stats[model_1]['ties'] += 1
        model_stats[model_2]['ties'] += 1
    else:
        model_stats[model_1]['losses'] += 1
        model_stats[model_2]['wins'] += 1

# Calculate win rate for each model
win_rates = {model: stats['wins'] / stats['total'] if stats['total'] > 0 else 0 for model, stats in model_stats.items()}

# Sort models by win rate in descending order
sorted_win_rates = sorted(win_rates.items(), key=lambda item: item[1], reverse=True)

# Print the sorted win rates
print("Sorted model win rates:")
for rank, (model, win_rate) in enumerate(sorted_win_rates, start=1):
    print(f"{rank}. {model}: Win Rate = {win_rate:.2f}")



Sorted model win rates:
1. RAG gpt-4 Answer: Win Rate = 0.88
2. RAG gpt-3.5-turbo Answer: Win Rate = 0.75
3. gpt-4 Answer: Win Rate = 0.66
4. gpt-3.5-turbo Answer: Win Rate = 0.40
5. Fine Tuned Mistral Answer: Win Rate = 0.30
6. Fine Tuned Biomistral DARE Answer: Win Rate = 0.27
7. Vanilla Biomistral DARE Answer: Win Rate = 0.18
8. Vanilla Mistral Answer: Win Rate = 0.11
